In [1]:
# Importing the Libraries

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import scipy.io.wavfile as wavfile
import scipy.fftpack as fftpk
from matplotlib import pyplot as plt

import sounddevice as sd
import soundfile as sf
import librosa

from IPython.display import Image
import ipywidgets as widgets
from IPython.display import display
    

#This Function train the data for a given fault ID
def training(faultID):
    if (faultID== None):
        pass
    else:
        # Creating a Dataframe to store the results
        input_test = pd.DataFrame(np.zeros((2,9)))
        input_test.columns = ["RMS","Mean","MA1","MA2","MA3","F1","F2","F3","FailureID"]
    
        # Training the data repeatively
        for i in range(4):
            
            # Recording the audio
            samplerate = 48000  
            duration = 5 # seconds
            audio = 'testing.wav'
            print("training data",i+1,"....")
            mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,channels=1, blocking=True)
            sd.wait()
            sf.write(audio, mydata, samplerate)
            
            # Reading the audio file
            s_rate, signal = wavfile.read(audio)
            
            # Extracting the features from time domain
            mean = abs(signal).mean()
            rms = np.sqrt(abs((signal**2).mean()))
            
            # Extracting the feaures from frequency domain with Fast Fourier Transform
            FFT_ = abs(fftpk.fft(signal))
            FFT = FFT_[range(len(FFT_)//2)]
            freqs = fftpk.fftfreq(len(FFT), 1.0/s_rate)[range(len(FFT_)//2)]
            
            # Extracting the 3 highest amplitudes in freqeuency spectrum 
            sorted = np.sort(FFT)[::-1]
            ma1, ma2 , ma3 = sorted[[0,1,2]]
    
            # Exrtacting the frequency positions of above amplitudes
            max_amp_index = [np.where(FFT == sorted[0])[0], np.where(FFT == sorted[1])[0], np.where(FFT == sorted[2])[0]]
            f1, f2, f3 = np.concatenate(freqs[max_amp_index])
    
            # Adding all the extracted features into the dataframe
            features = [rms,mean,ma1,ma2,ma3,f1,f2,f3,faultID]
            input_test.loc[i]= features
           
        print('Successfully trained the faultID',faultID)    
        return input_test
     
            
        

In [2]:
def trainAll(ready):
    
   
    IDall= pd.DataFrame()
    faultID=1
        while True:
            ID = training(faultID)
            Ready = widgets.Checkbox(False, description='Start next')
            Not_ready = widgets.Checkbox(False, description='Stop')
            if Ready.value == True:
                IDall= pd.concat([IDall,ID],ignore_index=True)
                break
            elif Not_ready.value == True:
                IDall= pd.concat([IDall,ID],ignore_index=True)
                faultID +=1
        IDall.to_csv("data/failure_classification_training.csv",index=False)
        data = pd.read_csv("data/failure_classification_training.csv")
        return data 
        

In [36]:
IDall = trainAll()
IDall

training data 1 ....
training data 2 ....
training data 3 ....
training data 4 ....
Successfully trained the faultID 1


All fault IDs are trained? (Type "Yes" or "No") No


training data 1 ....
training data 2 ....
training data 3 ....
training data 4 ....
Successfully trained the faultID 2


All fault IDs are trained? (Type "Yes" or "No") No


training data 1 ....
training data 2 ....
training data 3 ....
training data 4 ....
Successfully trained the faultID 3


All fault IDs are trained? (Type "Yes" or "No") Yes


,RMS,Mean,MA1,MA2,MA3,F1,F2,F3,FailureID
0,2.371729,1.862942,70582.201055,44957.938059,42464.528787,2.8,1.2,5.6,1.0
1,2.325790,1.825933,35662.282358,34522.145758,27732.243961,6.4,2.0,8.4,1.0
2,2.320941,1.823533,38012.689287,37834.148537,31264.074340,2.8,6.0,2.4,1.0
3,2.332870,1.833342,40215.863164,34676.628611,28908.147465,4.8,7.6,4.0,1.0
4,2.378052,1.861967,61710.233255,49222.121476,40121.539909,4.0,3.2,1.6,2.0
5,2.273188,1.784217,31818.292810,30500.732825,28245.946034,6.0,2.0,5.6,2.0
6,2.277147,1.788800,32449.675131,30182.210549,29478.038476,1.2,3.2,1.6,2.0
7,2.307180,1.811046,44864.188557,42049.738893,30576.481319,3.6,4.4,3.2,2.0
8,2.321391,1.824500,48901.823393,44747.661513,37204.854942,2.8,1.6,3.6,3.0
9,2.274521,1.784396,42369.151421,40951.170688,25898.628243,3.6,5.2,10.0,3.0


In [3]:
# Creating the recored button
train_button = widgets.Button(description='Train',disabled=False,button_style='primary',tooltip='Record',icon='play')
faultID = widgets.Text(description = "FaultID: ", disabled = False)
       
# Creating the recored button
record_button = widgets.Button(description='Predict',disabled=False,button_style='success',tooltip='Record',icon='bar-chart')

# Creating the stop button
stop_button = widgets.Button(description='Stop',disabled=False,button_style='warning',tooltip='Stop',icon='stop')

# Creating the output widget
output = widgets.Output()

# This function record the data and show the preictions
def start_recording(data):
    
    with output:
        output.clear_output()
        training(faultID)
    


train_button.on_click(start_recording)

# Displaying the output and buttons

x3 = widgets.Dropdown(description = "Model:",options=[("Option 1 - Classifier" ,0),("Option 2 - Anomaly Detection",1 )])
x2 = widgets.Text(description = "FaultID: ", disabled = False)
def x3_callback(change):
    x2.value ="{}".format(change["new"][0])
    
    
    
x3.observe(x3_callback,names="value")

display(x3) 
display(x2)



display(train_button)
b= widgets.HBox([record_button,stop_button])

b.layout.justify_content = 'center'

display(b)
display(output)

Dropdown(description='Model:', options=(('Option 1 - Classifier', 0), ('Option 2 - Anomaly Detection', 1)), va…

Text(value='', description='FaultID: ')

Button(button_style='primary', description='Train', icon='play', style=ButtonStyle(), tooltip='Record')

Output()

In [4]:
(train_button.on_click)

<bound method Button.on_click of Button(button_style='primary', description='Train', icon='play', style=ButtonStyle(), tooltip='Record')>

In [5]:
def trainingA(ready):
    
    if (ready==False):
        pass
    else:
        
        # Creating a Dataframe to store the results
        input_test = pd.DataFrame(np.zeros((2,8)))
        input_test.columns = ["RMS","Mean","MA1","MA2","MA3","F1","F2","F3"]
    
        # Training the data repeatively
        for i in range(2):
            
            # Recording the audio
            samplerate = 48000  
            duration = 5 # seconds
            audio = 'A_testing.wav'
            print("training data",i+1,"....")
            mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,channels=1, blocking=True)
            sd.wait()
            sf.write(audio, mydata, samplerate)
            
            # Reading the audio file
            s_rate, signal = wavfile.read(audio)
            
            # Extracting the features from time domain
            mean = abs(signal).mean()
            rms = np.sqrt(abs((signal**2).mean()))
            
            # Extracting the feaures from frequency domain with Fast Fourier Transform
            FFT_ = abs(fftpk.fft(signal))
            FFT = FFT_[range(len(FFT_)//2)]
            freqs = fftpk.fftfreq(len(FFT), 1.0/s_rate)[range(len(FFT_)//2)]
            
            # Extracting the 3 highest amplitudes in freqeuency spectrum 
            sorted = np.sort(FFT)[::-1]
            ma1, ma2 , ma3 = sorted[[0,1,2]]
    
            # Exrtacting the frequency positions of above amplitudes
            max_amp_index = [np.where(FFT == sorted[0])[0], np.where(FFT == sorted[1])[0], np.where(FFT == sorted[2])[0]]
            f1, f2, f3 = np.concatenate(freqs[max_amp_index])
    
            # Adding all the extracted features into the dataframe
            features = [rms,mean,ma1,ma2,ma3,f1,f2,f3]
            input_test.loc[i]= features
           
        print('Successfully trained')    
        return input_test

In [6]:
option = widgets.Dropdown(description = "Model:",options=[("Option 1 - Classifier" ,0),("Option 2 - Anomaly Detection",1 )],value=None)
IDall = pd.DataFrame()
def model_select(m):
    global IDall
    if m ==0:
        Ready = widgets.Checkbox(False, description='Start')
        widgets.interact(trainAll, ready=Ready);    
    elif m == 1:
        Ready = widgets.Checkbox(False, description='Start')
        widgets.interact(trainingA, ready=Ready);
widgets.interact(model_select, m=option);

interactive(children=(Dropdown(description='Model:', options=(('Option 1 - Classifier', 0), ('Option 2 - Anoma…

In [7]:
x= widgets.interact(training, faultID=FaultID);
IDall=pd.DataFrame()

ID = ((x.__dict__)['widget'].__dict__)["result"]
IDall = pd.concat([IDall,ID],igno)
IDall

NameError: name 'FaultID' is not defined

In [219]:
FaultID.value

3